In [1]:
# Importing Image class from PIL module 
from PIL import Image
import boto3

In [2]:
#returns the different labels of objects that are found in the photo
def detect_labels(photo, bucket):

    client=boto3.client('rekognition')

    response = client.detect_labels(Image={'S3Object':{'Bucket':bucket,'Name':photo}},
        MaxLabels=15)

    return (response['Labels'])

In [3]:
#checks if a person is detected in the photo and returns the objectbox values
def getPersonObjectBox(labels):

    for label in labels:
        if label['Name'] == "Person":
            for instance in label['Instances']:
                personObjectBox = {
                    "Type": "Person",
                    "width": instance['BoundingBox']['Width'],
                    "height": instance['BoundingBox']['Height'],
                    "top": instance['BoundingBox']['Top'],
                    "left": instance['BoundingBox']['Left'],
                }
                return personObjectBox
            else:
                return False

In [4]:
def cropImage(left, upper, right, lower, index, path):

    #change path to use \ for windows compatibility

    path = path.replace('/', '\\')

    # Opens a image in RGB mode 
    im = Image.open(f'C:\ANPR_FOTOS\Alarm Picture\{path}')

    im_crop = im.crop((left, upper, right, lower))

    # im_crop.show()

    im_crop.save(f'C:\ANPR_FOTOS\Alarm Picture\demo\cropped{index}.jpg', 'JPEG')

In [5]:
def calculateCoordinates(imageWidth, imageHeight, person):

    print(f'HIERE: {person}')

    #calculate the coördinates of the person objectBox 

    #calculate top/left (x,y) coördinate

    x1 = imageWidth * person['left']
    y1 = imageHeight * person['top']

    #calculate bottom/right (x,y) coördinate

    x2 = x1 + imageWidth * person['width']
    y2 = y1 + imageHeight * person['height']

    #add margins to get the windshield of the car left - 20%, right + 20%, top - 15% and bottom + 15%
    
    x1 = x1 - x1 * 0.2
    y1 = y1 - y1 * 0.15

    x2 = x2 + x2 * 0.2
    y2 = y2 + y2 * 0.15

    coordinates = makeImageSquare(x1, y1, x2, y2)

    return coordinates

In [6]:
#the goal of this function is to recalculate the coordinates so that the image will be cropped to a quare image of   244 x 244, because this is the default format for mobilenet
def makeImageSquare(x1, y1, x2, y2):
    
    print(x1, y1, x2, y2)

    currentWidth = x2 - x1 
    currentHeight = y2 - y1

    if(currentWidth > 244):
        #calculate the amount of pixel that we have to much and divide by 2 so we can add an equal amount on both sides
        extra = (currentWidth - 244) / 2
        x1 = x1 + extra
        x2 = x2 - extra
    else:
        extra = (244 - currentWidth) / 2
        x1 = x1 - extra
        x2 = x2 + extra

    if(currentHeight > 244):
        #calculate the amount of pixel that we have to much and divide by 2 so we can add an equal amount on both sides
        extra = (currentHeight - 244) / 2
        y1 = y1 + extra
        y2 = y2 - extra
    else:
        extra = (244 - currentHeight) / 2
        y1 = y1 - extra
        y2 = y2 + extra 

    coordinates = {
        "x1": x1,
        "y1": y1,
        "x2": x2,
        "y2": y2,
    }

    return coordinates



In [7]:
def main():
    # photo='Fotos-7-12-2020/20000101005741860_1_1SHP373_1_8.jpg'
    bucket='ai-essentials-anpr'

    # connect to the aws s3 bucket

    conn = boto3.client('s3')

    # loop over a list of all images in the s3 bucket and crop all windshields out of the photographs
    # enumerate is used to keep track of the index in the loop
    for index, key in enumerate(conn.list_objects(Bucket='ai-essentials-anpr')['Contents']):
        #key['Key'] is the name of a photo in the s3 bucket
        labels=detect_labels(key['Key'], bucket)

        person = getPersonObjectBox(labels)

        if(person):
            #image dimensions
            imageWidth = 1920
            imageHeight = 1144

            coordinates = calculateCoordinates(imageWidth, imageHeight, person)

            print(coordinates['x1'], coordinates['y1'])
            print(coordinates['x2'], coordinates['y2'])

            cropImage(coordinates['x1'], coordinates['y1'], coordinates['x2'], coordinates['y2'], index, key['Key'])
        else:
            print('No person detected')


    #some tests
    # labels=detect_labels('Fotos-7-12-2020/20000101005807043_1_1SHP373_1_21.jpg', bucket)

    # person = getPersonObjectBox(labels)

    # print(person)

    # coordinates = calculateCoordinates(1920, 1144, person)

    # print(coordinates)

    # image ='Fotos-7-12-2020/20000101005807043_1_1SHP373_1_21.jpg'

    # cropImage(coordinates['x1'], coordinates['y1'], coordinates['x2'], coordinates['y2'], 100001, image)    


if __name__ == "__main__":
    main()


HIERE: {'Type': 'Person', 'width': 0.06595519185066223, 'height': 0.12109780311584473, 'top': 0.1640293151140213, 'left': 0.5231375694274902}
803.539306640625 159.5021060168743 1357.2697219848633 375.11323674321176
958.4045143127441 145.30767138004302
1202.4045143127441 389.307671380043
HIERE: {'Type': 'Person', 'width': 0.0904439315199852, 'height': 0.13863599300384521, 'top': 0.129844531416893, 'left': 0.522415280342102}
802.4298706054688 126.26082234978676 1412.027624130249 353.2129779279232
985.2287473678589 117.73690013885498
1229.228747367859 361.736900138855
HIERE: {'Type': 'Person', 'width': 0.07100117951631546, 'height': 0.1280696839094162, 'top': 0.16073989868164062, 'left': 0.5073753595352173}
779.3285522460938 156.30347747802733 1332.5795459747314 379.95788685679435
933.9540491104126 146.13068216741084
1177.9540491104126 390.13068216741084
HIERE: {'Type': 'Person', 'width': 0.10834947973489761, 'height': 0.15524432063102722, 'top': 0.13076844811439514, 'left': 0.51708680391

KeyboardInterrupt: 